# Relationship between marijuana/cannabis use and household income 

   Many studies have found a relationship between substance abuse and socioeconomic background, showing that substance abuse in young adults/children is associated with lower socioeconomic status. The data set we chose to help understand these associations is a survey done in 2018/2019 regarding student tobacco, alcohol and drug use. We want to find out if there exists a relationship between marijuana/cannabis use and household income.

Methods: 
How will we conduct data analysis and which variables will we use?

**CAN_040** : In the last 30 days, how often did you use marijuana or cannabis?	
**DVHHINC2** : Median Household Income of the area where the respondent’s school is located according to the Canadian 2016 census data

**Describe one way we will visualize the results**
Bar plot: To display the relationship between income and number of people using marijuana, we will plot income on the x-axis and number of drug users on the y-axis. 
	

**Expected Outcomes and Significance**
	We expect to find that if household income is higher, then marijuana/cannabis use would be lower, so a negative relationship between these two variables.

**what impact will these findings have?**
If our findings show that a particular income group is associated with drug use, organizations would be able to address particular income areas in an attempt to lower substance abuse.

If our hypothesis is correct, we could address further questions such as :
Would individuals who come from a lower income household have better access to other types of drugs? 
Why would higher income individuals have lower drug use?
What other reasoning would be behind higher drug use among individuals who come from lower income households? Is bullying a factor?
